In [ ]:
import azure.cognitiveservices.speech as speechsdk

def speech_to_text(audio_file):
    subscription_key = "FKnTVDn6dNgyjJ5JtuSHKUylfwCFT5NOLIHTuIvABp7HDgia5PmFJQQJ99BCACYeBjFXJ3w3AAAAACOG8KSL"
    region = "eastus"
    speech_config = speechsdk.SpeechConfig(subscription=subscription_key, region=region)
    speech_config.speech_recognition_language = "th-TH"
    
    # Set maximum duration to 5 minutes (300 seconds)
    MAX_DURATION = 300  # 5 minutes in seconds
    done = False
    all_text = []

    def handle_result(evt):
        if evt.result.text:
            all_text.append(evt.result.text)

    def stop_cb(evt):
        nonlocal done
        done = True

    def canceled_cb(evt):
        nonlocal done
        done = True

    audio_config = speechsdk.audio.AudioConfig(filename=audio_file)
    speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_config)

    # Connect callbacks
    speech_recognizer.recognized.connect(handle_result)
    speech_recognizer.session_stopped.connect(stop_cb)
    speech_recognizer.canceled.connect(canceled_cb)

    # Start continuous recognition with timeout
    speech_recognizer.start_continuous_recognition()
    
    import time
    start_time = time.time()
    
    while not done and (time.time() - start_time) < MAX_DURATION:
        time.sleep(0.5)
    
    speech_recognizer.stop_continuous_recognition()
    
    final_text = " ".join(all_text)
    if final_text:
        return final_text
    else:
        return "Speech recognition failed or no speech detected"

Recognized Text: สวัสดีครับ โทรมาจาก สภ อําเภอคลองหลวง


In [11]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

def analyze_text_for_scam(text):
    llm = ChatOpenAI(
        openai_api_key="sk-GqA4Uj6iZXaykbOzIlFGtmdJr6VqiX94NhhjPZaf81kylRzh",
        openai_api_base="https://api.opentyphoon.ai/v1",
        model_name="typhoon-v2-70b-instruct",
    )
    template = """
        วิเคราะห์ข้อความต่อไปนี้เพื่อหาสัญญาณที่อาจเป็นการหลอกลวง:
        {text}

        กรุณาระบุ:
        1. วลีหรือรูปแบบที่น่าสงสัย
        2. ระดับความเสี่ยง (ต่ำ/กลาง/สูง)
        3. ประเภทของการหลอกลวงที่อาจเกิดขึ้น (ถ้ามี)
        4. เหตุผลสำหรับการประเมิน

        การตอบกลับ:
    """
    
    prompt = PromptTemplate(
        input_variables=["text"],
        template=template
    )
    
    chain = prompt | llm
    response = chain.invoke({"text": text})
    
    return response.content

speech_text = text
analysis_result = analyze_text_for_scam(speech_text)
print("\nScam Analysis Result:")
print(analysis_result)


Scam Analysis Result:
1. วลีหรือรูปแบบที่น่าสงสัย: "โทรมาจาก สภ อําเภอคลองหลวง"
2. ระดับความเสี่ยง: กลาง
3. ประเภทของการหลอกลวงที่อาจเกิดขึ้น: การหลอกลวงทางโทรศัพท์
4. เหตุผลสำหรับการประเมิน: ข้อความดังกล่าวอาจเป็นการหลอกลวงเนื่องจากการใช้ชื่อสถานที่ราชการ เช่น "สภ อําเภอคลองหลวง" เพื่อสร้างความน่าเชื่อถือ แต่ไม่มีข้อมูลเพิ่มเติมเกี่ยวกับเหตุผลในการโทรศัพท์ หรือข้อมูลที่ชัดเจนเกี่ยวกับเรื่องที่ต้องการพูดคุย นอกจากนี้ การโทรศัพท์จากหน่วยงานราชการโดยตรงอาจมีการแจ้งให้ทราบล่วงหน้าหรือมีข้อมูลที่ชัดเจนมากขึ้น ดังนั้นจึงควรใช้ความระมัดระวังและตรวจสอบข้อมูลก่อนการตอบสนอง
